In [30]:
import pandas as pd
import numpy as np
from itertools import product

In [33]:
parameters = {"heap": ["-Xms{}m -Xmx{}m".format(v, v) for v in [512, 1024, 2048, 4096]],
              "gc": ["-XX:+{}".format(v) for v in ["UseSerialGC", "UseParallelGC", "UseConcMarkSweepGC", "UseG1GC"]]
             }

In [62]:
import collections
statistics = []
for d in [dict(zip(parameters, v)) for v in product(*parameters.values())][:3]:
    heap_config, gc = d["gc"], d["heap"]
    !java $heap_config $gc -Xloggc:gc.log -XX:+PrintGCDetails -XX:+PrintGCDateStamps GCLogAnalysis > objects.log
    statistic = d

    with open("gc.log", "r") as file:
        lines = file.readlines()
        statistic['log'] = lines
    with open("objects.log", "r") as file:
        statistic['throughput'] = int("".join(file.readlines()).split(":")[1].strip())
    statistics.append(statistic)


In [63]:
df = pd.DataFrame(statistics)
df

,gc,heap,log,throughput
0,-XX:+UseSerialGC,-Xms512m -Xmx512m,[Java HotSpot(TM) 64-Bit Server VM (25.162-b12...,9458
1,-XX:+UseParallelGC,-Xms512m -Xmx512m,[Java HotSpot(TM) 64-Bit Server VM (25.162-b12...,7969
2,-XX:+UseConcMarkSweepGC,-Xms512m -Xmx512m,[Java HotSpot(TM) 64-Bit Server VM (25.162-b12...,9553


## Evaluate Metrics
- Full GC times
- Young GC times
- Full GC STW duration
- Young GC STW duration
- Throughput 

In [54]:
df[df['gc'] == '-XX:+UseSerialGC'].reset_index(drop=True).iloc[0]['log']

['Java HotSpot(TM) 64-Bit Server VM (25.162-b12) for bsd-amd64 JRE (1.8.0_162-b12), built on Dec 19 2017 17:33:35 by "java_re" with gcc 4.2.1 (Based on Apple Inc. build 5658) (LLVM build 2336.11.00)\n',
 'Memory: 4k page, physical 16777216k(1861428k free)\n',
 '\n',
 '/proc/meminfo:\n',
 '\n',
 'CommandLine flags: -XX:InitialHeapSize=536870912 -XX:MaxHeapSize=536870912 -XX:+PrintGC -XX:+PrintGCDateStamps -XX:+PrintGCDetails -XX:+PrintGCTimeStamps -XX:+UseCompressedClassPointers -XX:+UseCompressedOops -XX:+UseSerialGC \n',
 '2020-10-26T01:26:29.066+0500: 0.153: [GC (Allocation Failure) 2020-10-26T01:26:29.066+0500: 0.153: [DefNew: 139776K->17471K(157248K), 0.0302487 secs] 139776K->51342K(506816K), 0.0303461 secs] [Times: user=0.02 sys=0.01, real=0.03 secs] \n',
 '2020-10-26T01:26:29.120+0500: 0.207: [GC (Allocation Failure) 2020-10-26T01:26:29.120+0500: 0.207: [DefNew: 157247K->17467K(157248K), 0.0356831 secs] 191118K->93841K(506816K), 0.0357644 secs] [Times: user=0.02 sys=0.01, real=0.